In [3]:
# Bitcoin Analysis and Visualization Script - Version 013
# This version includes:
# - Bar charts for metrics visualization with price on left y-axis and indicators on right y-axis
# - Heatmap for overall sentiment analysis with improved styling
# - Removal of grid lines from all charts
# - Adjusted and aligned heatmap color legend
# - Saving all charts to a single HTML file with white background

import requests
import pandas as pd
import numpy as np
import io
from datetime import datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Constants
API_KEY = '2lZRFGaqFiEYkzr7WUuT4EaoC1X'  # Replace with your actual API key
SINCE_DATE = int(datetime(2023, 1, 1).timestamp())  # Jan 1, 2023
UNTIL_DATE = int(datetime.now().timestamp())  # Current date

# URLs for fetching data
PRICE_URL = 'https://api.glassnode.com/v1/metrics/market/price_usd_close'
METRICS = [
    'https://api.glassnode.com/v1/metrics/market/spot_cvd_sum',
    'https://api.glassnode.com/v1/metrics/market/spot_volume_daily_sum'
]

def fetch_glassnode_data(url, asset='BTC'):
    params = {
        'a': asset,
        's': SINCE_DATE,
        'u': UNTIL_DATE,
        'api_key': API_KEY,
        'f': 'CSV',
        'c': 'USD'
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        df = pd.read_csv(io.StringIO(response.text))
        metric_name = url.split('/')[-1]
        df.columns = ['t', metric_name]
        df['t'] = pd.to_datetime(df['t'], unit='s')
        df[metric_name] = pd.to_numeric(df[metric_name], errors='coerce')
        return df
    else:
        print(f"Failed to fetch data from {url}. Status code: {response.status_code}")
        return None

# Fetch and merge data
price_df = fetch_glassnode_data(PRICE_URL)
all_dfs = [price_df]
for metric_url in METRICS:
    metric_df = fetch_glassnode_data(metric_url)
    if metric_df is not None:
        all_dfs.append(metric_df)

merged_df = pd.concat(all_dfs, axis=1)
merged_df = merged_df.loc[:,~merged_df.columns.duplicated()]
merged_df.set_index('t', inplace=True)

def calculate_momentum_rsi(df, column='price_usd_close', rsi_window=14, window_norm=90, normalize=True):
    price_change = df[column].diff()
    gains = price_change.where(price_change > 0, 0)
    losses = -price_change.where(price_change < 0, 0)
    avg_gains = gains.rolling(window=rsi_window, min_periods=1).mean()
    avg_losses = losses.rolling(window=rsi_window, min_periods=1).mean()
    relative_strength = avg_gains / avg_losses
    rsi = 100 - (100 / (1 + relative_strength))

    if normalize:
        rsi_min = rsi.rolling(window=window_norm, min_periods=1).min()
        rsi_max = rsi.rolling(window=window_norm, min_periods=1).max()
        normalized_momentum = 2 * (rsi - rsi_min) / (rsi_max - rsi_min) - 1
        return normalized_momentum
    else:
        return rsi

def calculate_spot_cvd_bias(df, column='spot_cvd_sum', window_sum=7, window_norm=90, normalize=True):
    rolling_sum = df[column].rolling(window=window_sum).sum()
    
    if normalize:
        rolling_min = rolling_sum.rolling(window=window_norm, min_periods=1).min()
        rolling_max = rolling_sum.rolling(window=window_norm, min_periods=1).max()
        normalized_bias = 2 * (rolling_sum - rolling_min) / (rolling_max - rolling_min) - 1
        return normalized_bias
    else:
        return rolling_sum

def calculate_spot_volume_momentum(df, column='spot_volume_daily_sum', fast_window=7, slow_window=90, window_norm=90, normalize=True):
    fast_ma = df[column].rolling(window=fast_window).mean()
    slow_ma = df[column].rolling(window=slow_window).mean()
    volume_momentum = fast_ma / slow_ma

    if normalize:
        rolling_min = volume_momentum.rolling(window=window_norm, min_periods=1).min()
        rolling_max = volume_momentum.rolling(window=window_norm, min_periods=1).max()
        normalized_momentum = 2 * (volume_momentum - rolling_min) / (rolling_max - rolling_min) - 1
        return normalized_momentum
    else:
        return volume_momentum

# Apply the functions to our merged_df
merged_df['Price Momentum'] = calculate_momentum_rsi(merged_df, column='price_usd_close', rsi_window=14, window_norm=90, normalize=True)
merged_df['Spot CVD Bias'] = calculate_spot_cvd_bias(merged_df, column='spot_cvd_sum', window_sum=7, window_norm=90, normalize=True)
merged_df['Spot Volume Momentum'] = calculate_spot_volume_momentum(merged_df, column='spot_volume_daily_sum', fast_window=7, slow_window=90, window_norm=90, normalize=True)

def create_chart(df, metric_name, chart_title):
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add trace for BTC price (now on the primary y-axis)
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['price_usd_close'],
            mode='lines',
            line=dict(color='gray', width=1),
            name='BTC price in $'
        ),
        secondary_y=False,
    )

    # Add bar chart for the metric (now on the secondary y-axis)
    fig.add_trace(
        go.Bar(
            x=df.index,
            y=df[metric_name],
            name=metric_name,
            marker_color=df[metric_name].apply(lambda x: 'rgba(0,255,0,0.6)' if x >= 0 else 'rgba(255,0,0,0.6)'),
            marker_line_width=0  # This removes the outline of the bars
        ),
        secondary_y=True,
    )

    # Configure layout
    fig.update_layout(
        title={
            'text': chart_title,
            'font': {'color': 'grey'}
        },
        plot_bgcolor='white',
        paper_bgcolor='white',
        xaxis=dict(
            title='Date',
            titlefont={'color': 'grey'},
            showgrid=False,
            showline=True,
            linewidth=1,
            linecolor='grey',
            ticks='outside',
            ticklen=5,
            tickwidth=1,
            tickcolor='grey',
            tickfont={'color': 'grey'},
            tickformat='%b %y',
            tickmode='auto',
            nticks=10,
            mirror=True
        ),
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=1.02,
            xanchor='right',
            x=1,
            font={'color': 'grey'}
        ),
        hovermode='x unified',
    )

    # Update y-axes
    fig.update_yaxes(
        title='BTC price in $',
        titlefont={'color': 'grey'},
        showgrid=False,
        showline=True,
        linewidth=1,
        linecolor='grey',
        ticks='outside',
        ticklen=5,
        tickwidth=1,
        tickcolor='grey',
        tickfont={'color': 'grey'},
        mirror=True,
        secondary_y=False
    )

    fig.update_yaxes(
        title=metric_name,
        titlefont={'color': 'grey'},
        range=[-1, 1],
        showgrid=False,
        zeroline=True,
        zerolinecolor='rgba(0,0,0,0.2)',
        tickmode='array',
        tickvals=[-1, -0.5, 0, 0.5, 1],
        ticktext=['-1', '-0.5', '0', '0.5', '1'],
        ticks='outside',
        ticklen=5,
        tickwidth=1,
        tickcolor='grey',
        tickfont={'color': 'grey'},
        showline=True,
        linewidth=1,
        linecolor='grey',
        mirror=True,
        secondary_y=True
    )

    return fig

def create_heatmap(df):
    indicators = ['Price Momentum', 'Spot CVD Bias', 'Spot Volume Momentum']
    
    fig = go.Figure(data=go.Heatmap(
        z=[df[indicator] for indicator in indicators],
        x=df.index,
        y=indicators,
        colorscale=[
            [0, 'red'],      # Negative Momentum (-1)
            [0.5, 'yellow'], # Neutral Momentum (0)
            [1, 'green']     # Positive Momentum (+1)
        ],
        zmin=-1,
        zmax=1,
        colorbar=dict(
            title='Momentum',
            titleside='right',
            tickvals=[-1, 0, 1],
            ticktext=['Negative Momentum', 'Neutral Momentum', 'Positive Momentum'],
            ticks='outside',
            tickfont=dict(color='grey'),
            titlefont=dict(color='grey'),
            len=0.5,  # Reduce the length of the colorbar
            y=0.5,    # Center the colorbar vertically
        )
    ))

    fig.update_layout(
        title={
            'text': 'Spot Market Sentiment Heatmap',
            'font': {'color': 'grey'}
        },
        plot_bgcolor='white',
        paper_bgcolor='white',
        xaxis=dict(
            title='Date',
            titlefont={'color': 'grey'},
            tickfont={'color': 'grey'},
            tickformat='%b %y',
            tickmode='auto',
            nticks=10,
            showgrid=False,
        ),
        yaxis=dict(
            title='Indicators',
            titlefont={'color': 'grey'},
            tickfont={'color': 'grey'},
            tickmode='array',
            tickvals=[0, 1, 2],
            ticktext=indicators,
            showgrid=False,
        ),
        height=200,  # Reduced height to make indicator traces narrower
        margin=dict(l=50, r=50, t=50, b=50)  # Adjust margins as needed
    )

    return fig

# Create a list to store all figures
all_figures = []

# Create and store the charts
charts = [
    ("Price Momentum", "Bitcoin Price and Price Momentum"),
    ("Spot CVD Bias", "Bitcoin Price and Spot CVD Bias"),
    ("Spot Volume Momentum", "Bitcoin Price and Spot Volume Momentum")
]

for metric, title in charts:
    fig = create_chart(merged_df, metric, title)
    all_figures.append(fig)
    fig.show()

# Create and store the heatmap
heatmap = create_heatmap(merged_df)
all_figures.append(heatmap)
heatmap.show()

# Create a single HTML file with all charts
dashboard = make_subplots(rows=4, cols=1, vertical_spacing=0.1,
                          subplot_titles=[fig.layout.title.text for fig in all_figures],
                          specs=[[{"secondary_y": True}]] * 4)

for i, fig in enumerate(all_figures, start=1):
    for trace in fig.data:
        dashboard.add_trace(trace, row=i, col=1, secondary_y=trace.yaxis == 'y2')

dashboard.update_layout(
    height=2400, 
    title_text="Bitcoin Market Sentiment Analysis Dashboard",
    title_font=dict(size=24, color='grey'),  # Increased size for main title
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(size=14, color='grey')  # Increased base font size
)

# Update layout for each subplot
for i in range(1, 5):
    dashboard.update_xaxes(title_text="Date", row=i, col=1, 
                           titlefont=dict(size=16, color='grey'),  # Increased size for axis titles
                           tickfont=dict(size=12, color='grey'),  # Increased size for tick labels
                           showgrid=False)
    dashboard.update_yaxes(title_text="BTC price in $", 
                           titlefont=dict(size=16, color='grey'),  # Increased size for axis titles
                           tickfont=dict(size=12, color='grey'),  # Increased size for tick labels
                           row=i, col=1, secondary_y=False,
                           showgrid=False)
    if i < 4:  # Don't apply to the heatmap
        dashboard.update_yaxes(title_text=charts[i-1][0], 
                               titlefont=dict(size=18, color='grey'),  # Increased size for axis titles
                               tickfont=dict(size=18, color='grey'),  # Increased size for tick labels
                               row=i, col=1, secondary_y=True,
                               range=[-1, 1], tickmode='array', tickvals=[-1, -0.5, 0, 0.5, 1],
                               ticktext=['-1', '-0.5', '0', '0.5', '1'],
                               showgrid=False)

# Adjust the colorbar for the heatmap
dashboard.update_layout(
    coloraxis_colorbar=dict(
        title='Momentum',
        titleside='right',
        tickvals=[-1, 0, 1],
        ticktext=['Negative Momentum', 'Neutral Momentum', 'Positive Momentum'],
        ticks='outside',
        tickfont=dict(size=12, color='grey'),  # Increased size for colorbar tick labels
        titlefont=dict(size=16, color='grey'),  # Increased size for colorbar title
        len=0.75,
        thickness=15,
        yanchor='bottom',
        y=0.03,
        xanchor='right',
        x=1.0
    )
)

# Update subplot titles
for i in dashboard['layout']['annotations']:
    i['font'] = dict(size=18, color='grey')  # Increased size for subplot titles

# Save the dashboard to an HTML file
pio.write_html(dashboard, file='bitcoin_sentiment_dashboard.html', auto_open=True)

print("All charts have been displayed and saved to 'bitcoin_sentiment_dashboard.html'")

All charts have been displayed and saved to 'bitcoin_sentiment_dashboard.html'


In [4]:
merged_df.tail()

,price_usd_close,spot_cvd_sum,spot_volume_daily_sum,Price Momentum,Spot CVD Bias,Spot Volume Momentum
t,,,,,,
2024-09-20,63122.179999,-3.372853e+07,8.501245e+09,0.825269,0.250265,-0.435338
2024-09-21,63486.442220,-1.418283e+07,3.468778e+09,0.828765,0.313421,-0.453242
2024-09-22,63567.370107,8.380872e+06,5.078147e+09,0.801168,0.383459,-0.412888
2024-09-23,63322.551391,8.604391e+07,8.047968e+09,0.653095,0.574273,-0.408427
2024-09-24,64338.712500,-1.330203e+08,7.888459e+09,0.679914,0.383670,-0.474315
